All notebooks must have a cell with the `//default_exp <module_name>` tag. This tag will generate a Scala script `child.sc` in the `lib/` folder.

In [ ]:
//default_exp child

## Importing code from other notebooks
> In `00_parent.ipynb` we defined an `Add` module in Chisel. By running `chisel_nbdev_build_lib` we generated the file `parent.sc` to the destination `lib`. (To change this destination, modify `lib_name` in `settings.ini` to something other than `lib`.

We can import the full contents of the Scala script using the following notation:
```Scala
import $file.^.lib.parent, parent._
```

In [ ]:
//hide
//export
import $file.^.lib.parent, parent._

Compiling /Users/jasonvranek/code/notebooks/chisel_nbdev_template/lib/parent.sc

import $file.$           , parent._

Alternatively you can import like
```Scala
import $file.^.lib.parent
```
and use like `parent.Add`

### Load the Chisel dependencies from resource.chisel_deps.sc
> The exact version of Chisel3 and testers are defined in resources/chisel_deps.sc

In [ ]:
//export
import $file.^.resource.chisel_deps, chisel_deps._

import $file.$                     , chisel_deps._

To resolve namespace issue between Almond and Chisel's Input, make sure to import Input like:
```Scala
import chisel3.{Input => Input}
```


In [ ]:
//hide
//export
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.test
import chisel3.tester._

import chisel3.{Input => Input}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3.tester.RawTester.test

import chisel3.tester._

This test should compile and pass as we brought `Add` into the notebook scope.

In [ ]:
test(new Add) { c =>
    c.io.a.poke(1.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(3.U)
    
    c.io.a.poke(5.U)    
    c.io.b.poke(2.U)    
    c.clock.step(1)    
    c.io.out.expect(7.U)
}

[info] [0.001] Elaborating design...
[info] [1.227] Done elaborating.
file loaded in 0.033681041 seconds, 7 symbols, 3 statements
test Add Success: 0 tests passed in 4 cycles in 0.042766 seconds 93.53 Hz


# ComposedModule

> We can use the Add module now to build more complex modules.

In [ ]:
//export
class ComposedModule extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(4.W))
        val b = Input(UInt(4.W))
        val c = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    val addr = Module(new Add)
    addr.io.a := io.a
    addr.io.b := io.b
    io.out := addr.io.out + io.c
}

defined class ComposedModule

Test the module using `chisel3.iotesters.PeekPokeTester`.

In [ ]:
assert(Driver(() => new ComposedModule) {
    c => new PeekPokeTester(c) {    
        poke(c.io.a, 1)    
        poke(c.io.b, 2)
        poke(c.io.c, 3)
        expect(c.io.out, 6)

        poke(c.io.a, 5)    
        poke(c.io.b, 2)    
        poke(c.io.c, 7)    
        expect(c.io.out, 14)
    }
})

[info] [0.000] Elaborating design...
[info] [0.078] Done elaborating.
Total FIRRTL Compile Time: 67.9 ms
Total FIRRTL Compile Time: 45.1 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1615354558172
test ComposedModule Success: 2 tests passed in 5 cycles taking 0.007856 seconds
[info] [0.006] RAN 0 CYCLES PASSED
